TODO

- results/gmark/rdflib: different results (slower) compared to June

In [1]:
import os, subprocess, re, multiprocess
from datetime import datetime

In [2]:
def run(cmd):    
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    
    if error.strip() != "":
        print(error)

    print(out)

def __do_runNSave(cmd, out_path, ret_times, get_times_eye=True):
     # result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.rstrip()
    # print("out:", out)
    # print("error:", error)
        
    with open(out_path, 'w') as fh:
        fh.write(out)
    
    if "** ERROR **" in error:
        print("ERROR:", error)
    
    elif get_times_eye:
        netw_time = int(re.search("networking \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        reas_time = int(re.search("reasoning \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        ret_times.append(netw_time); ret_times.append(reas_time)
        # return netw_time, reas_time

def runNSave_timed(cmd, out_path, max_time, get_times_eye=True):
    mngr = multiprocess.Manager()
    ret_times = mngr.list()
    
    p = multiprocess.Process(target=__do_runNSave, args=(cmd, out_path, ret_times, get_times_eye))
    p.start()
    
    p.join(max_time)
    if p.is_alive():
        p.kill()
        return (-1, -1)
    else:
        return ret_times
        
def runNSave(cmd, out_path, get_times_eye=True):
   ret_times = []
   __do_runNSave(cmd, out_path, ret_times, get_times_eye)
   
   return ret_times

def record_eye(times_file, run, query, data, type, phase, netw_time, reas_time):
    times_file.write(f"{run},{query},{data},{type},{phase},{netw_time},{reas_time}\n")
    
def record_sparql(times_file, run, query, data, load_time, exec_time):
    times_file.write(f"{run},{query},{data},{load_time},{exec_time}\n")

## Convert N3 to NT

In [ ]:
# convert to NT, replace first/rest pairs

folder = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/zika/data/new"

for file in os.listdir(folder):
    if not file.endswith(".n3"):
        continue
    
    print(file)
    
    name = file[:file.index(".")]
    nt_file = f"{name}.nt"
    
    # convert to NT
    
    path = os.path.join(folder, file)
    args = ['java', '-jar', "../test/run/turtle2nt.jar", "-turtle", path]
    process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [b.decode('UTF-8') for b in process.communicate()]
    
    # rename first/rest to avoid eye builtins
    # NOTE: rules should also use f1rst/r3st!
    
    out = out.replace("<http://www.w3.org/1999/02/22-rdf-syntax-ns#first>", "<http://www.w3.org/1999/02/22-rdf-syntax-ns#f1rst>")
    out = out.replace("<http://www.w3.org/1999/02/22-rdf-syntax-ns#rest>", "<http://www.w3.org/1999/02/22-rdf-syntax-ns#r3st>")
    
    with open(os.path.join(folder, nt_file), 'w') as nt_fh:
        nt_fh.write(out)

In [ ]:
# "freeze" bnodes

folder = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/zika/data/new"
for file in os.listdir(folder):
    if not file.endswith(".nt"):
        continue
    
    print(file)
    with open(os.path.join(folder, file), 'r') as nt_fh:
        contents = nt_fh.read()
    
    matches = [ (m.start(0), m.end(0)) for m in re.finditer("(_:(.*?)) ", contents) ]    
    array = []; prior = 0
    for (start, end) in matches:
        if start > 0:
            array.extend(contents[prior : start])
        
        id = contents[start+2 : end-1]
        repl = f"<http://example.org/{id}>"
        array.extend(repl)
        
        prior = end-1
        
    array.extend(contents[prior : len(contents)])
    
    contents = "".join(array)
    
    with open(os.path.join(folder, file), 'w') as nt_fh:
        nt_fh.write(contents)

## Convert SPARQL PP into N3 PP

In [ ]:
# NOTE for now run with Python 3.11.5 (manually fixed issue with NegatedPropertySet & reverse paths)

from rdflib.plugins.sparql import parser
from convert_pp_sparql_n3 import To_N3_Visitor

In [ ]:
# if someone else is running this notebook:

# @rdflib.plugins.sparql.parserutils.py#279
# add:
# elif isinstance(t,CompValue) or isinstance(t,URIRef):
#     res['part'] = t

### Convert

In [ ]:
# convert single query

# query = "SELECT ?x WHERE { ?x (:p1/:p2)* ?z ; !(:p3|:p4|:p5) ?a }"
# query = "PREFIX : <http://example.org/gmark/> " + \
#     "SELECT * WHERE { ?x0 !(^:p1|:p2) ?x3 } "
#     # "SELECT * WHERE { ?x0 ((^:p1/:p2*)?/:p3)+ ?x3 } "
#     # "SELECT * WHERE { ?x0 !(:p1|^:p2|:p3) ?x3 }"
query = """PREFIX : <http://example.org/gmark/> 
SELECT DISTINCT ?x2 ?x1 ?x3 ?x0 
WHERE {  {  ?x0 (((^:p1/:p1/^:p1/:p1)))* ?x1 . ?x1 ((^:p1/:p1/^:p1)|(^:p1/:p1/^:p1)) ?x2 . ?x2 (((:p1/^:p1)))+ ?x3 . } }
"""

query = parser.parseQuery(query)
query = query[1]

print(To_N3_Visitor().convert(query))

In [ ]:
# convert query folder

import os

visitor = To_N3_Visitor()

# path = "../../gmark_50_new/mix"
path = "zika"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    print(file)
    with open(os.path.join(path, file), 'r') as fh:
        query = fh.read()
        query = parser.parseQuery(query)
        query = query[1]
        
        conv = visitor.convert(query)
        # conv = "@prefix : <http://example.org/gmark/> .\n\n" + conv
        conv = "@prefix fh: <http://hl7.org/fhir/> .\n\n" + conv
        # print(conv)
        
        n3_file = file[0:file.index(".")] + ".n3"
        with open(os.path.join(path, "n3", n3_file), 'w') as fh2:
            fh2.write(conv)
        
        # print("\n")

## Run SPARQL PP

In [ ]:
import time
import rdflib
from rdflib.plugins.sparql.results.csvresults import CSVResultSerializer
from resCSV2N3 import convert as csv2n3

In [ ]:
# process = subprocess.Popen(["/Users/wvw/git/n3/eclipse-rdf4j-5.1.3/bin/console.sh"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
# out, error = [ b.decode('UTF-8') for b in process.communicate() ]

# for i in range(6):
#     print(process.stdout.readline())   

# print(process.communicate(input="create repo"))
# print(process.stdout.readline())

In [ ]:
def rdflib_sparql(query_file, data_file, result_file):
    start = time.perf_counter()
    g = rdflib.Graph()
    g.parse(data_file)
    
    query = open(query_file, 'r').read()
    end = time.perf_counter()
    load_time = (end - start) * 1000
    
    start = time.perf_counter()
    results = g.query(query)
    
    if "ASK" not in query:
        with open(result_file, 'wb') as fh:
            CSVResultSerializer(results).serialize(fh)
    else:
        result = str(bool(results)).lower()
        with open(result_file, 'w') as fh:
            fh.write(f"result: {result}")
    end = time.perf_counter()
    
    exec_time = (end - start) * 1000
    
    return load_time, exec_time

In [ ]:
def java_sparql(exec_path, query_file, data_file, result_file, qtype="select"):
    # print(data_file, query_file, result_file)
    
    args = ['java', '-jar', exec_path, "-n3", data_file, "-query", query_file, "-result", result_file]
    if exec_path == "../test/run/rdf4j_sparql.jar":
        args.extend(["-type", qtype])
        
    process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [b.decode('UTF-8') for b in process.communicate()]
    # if error.strip() != "":
    #     print(error)
    # else:
    out = str(out)
    # print(error)
    # print(out)
    load_time = re.search("load:(.*)", out).group(1)
    exec_time = re.search("exec:(.*)", out).group(1)
    return (load_time, exec_time)

def jena_sparql(query_file, data_file, result_file):
    return java_sparql("../test/run/jena_sparql.jar", query_file, data_file, result_file)

def rdf4j_sparql(query_file, data_file, result_file):
    qtype = "select"
    with open(query_file, 'r') as query_fh:
        if "ASK {" in query_fh.read():
            qtype = "ask"
    
    return java_sparql("../test/run/rdf4j_sparql.jar", query_file, data_file, result_file, qtype)

In [ ]:
# run query folder

# path = "../../gmark_50_new/mix"
# data_size = str(100)
# skip = open(f"skip/{data_size}/{system}.txt", 'r').read().splitlines()

path = "zika"
data_size = "7_000_pt2"
skip = []

system = 'jena'
nr_runs = 1

times_path = os.path.join(path, "results", system, data_size, f"times_{system}.csv")
exists = os.path.exists(times_path)
times_file = open(times_path, 'a')
if not exists:
    times_file.write("run,query,data,load_time,exec_time\n")
else:
    times_file.write("\n")

for run in range(0, nr_runs):
    print("run", ":", run)
    files = list(os.listdir(path))
    files.sort()
    for file in files:
        if not file.endswith(".sparql"):
            continue    
        if file in skip:
            continue
        
        print(file, ":", datetime.now().strftime("%H:%M:%S"))
        
        name = file[:file.index(".")]
        query_file = os.path.join(path, file)
        data_file = f"data_{data_size}.nt"
        data_path = os.path.join(path, "data", data_file)
        result_path_csv = os.path.join(path, "results", system, data_size, f"{name}.csv")
        result_path_n3 = os.path.join(path, "results", system, data_size, f"{name}.n3")
        
        match (system):
            case 'jena':
                sparql_fn = jena_sparql
            case 'rdflib':
                sparql_fn = rdflib_sparql
            case 'rdf4j':
                sparql_fn = rdf4j_sparql
        
        try:
            (load_time, exec_time) = sparql_fn(query_file, data_path, result_path_csv)
            record_sparql(times_file, run, file, data_file, load_time, exec_time)
            
            
            csv2n3(file=result_path_csv , ordered=False, out=result_path_n3)
            times_file.flush()
            
        except Exception as e:
            print(">> error <<", e)
    
    print()

## Run N3 PP

### Ground & normalize

In [8]:
# normalize entire folder

# path = "../../gmark_50_new/mix/n3"
path = "zika/n3"
for file in os.listdir(path):    
    if not file.endswith(".n3"):
        continue
    
    # if "pp4-re1" not in file:
    #     continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, file)
    norm_file = os.path.join(path, "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times_eye=False)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times_eye=False)

pp3.n3
pp4-re0.n3
pp2.n3
pp4-re1.n3
pp2-re0.n3
pp2-re1.n3
pp1.n3
pp4.n3
pp4-2.n3


### Run

In [ ]:
# run folder

# path = "../../gmark_50_new/mix"
# data_size = str(100)
# skip_direct = open(f'skip/{data_size}/n3_direct.txt', 'r').read().splitlines()
# skip_fwd = open(f'skip/{data_size}/n3_fwd.txt', 'r').read().splitlines()
# skip_bwd = open(f'skip/{data_size}/n3_bwd.txt', 'r').read().splitlines()

# for only_query in [ "pp4-re1.n3" ]:
for data_size in [ "1_000_pt2", "2_000_pt2", "5_000_pt2", "7_000_pt2", "10_000_pt2" ]:
    # print("setup", only_query, data_size)
    print("setup", data_size)
    
    path = "zika"
    # data_size = "10_000_pt2"
    skip_direct = []
    skip_bwd = []
    skip_fwd = []
    skip_mxd = []

    nr_runs = 1

    times_path = os.path.join(path, "n3", "results", data_size, "times_n3.csv")
    exists = os.path.exists(times_path)
    times_file = open(times_path, 'a')
    if not exists:
        times_file.write("run,query,data,type,phase,netw_time,reas_time\n")
    else:
        times_file.write("\n")

    for run in range(0, nr_runs):
        print("run:", run)
        
        files = list(os.listdir(os.path.join(path, "n3", "normalized")))
        files.sort()
        for file in files:        
            if not file.endswith(".n3"):
                continue
            
            # if only_query not in file:
            #     continue
            
            name = file[:file.index(".")]
            norm_file = os.path.join(path, "n3", "normalized", file)
            data_file = f"data_{data_size}.nt"
            data_path = os.path.join(path, "data", data_file)
            dir_res_file = os.path.join(path, "n3", "results", data_size, "direct", file)
            bwd_res_file = os.path.join(path, "n3", "results", data_size, "bwd", file)
            fwd_res_file = os.path.join(path, "n3", "results", data_size, "fwd", file)
            mxd_res_file = os.path.join(path, "n3", "results", data_size, "mxd", file)
            
            if not os.path.exists(norm_file):
                continue
            
            print(file, ":", datetime.now().strftime("%H:%M:%S"))
            
            max_time = 300
            
            try:
                # # - direct
                # if file not in skip_direct:
                #     netw_time, reas_time = runNSave_timed(["eye", "--turtle", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file, max_time=max_time)
                #     record_eye(times_file, run, file, data_file, 'direct', 'n/a', netw_time, reas_time)
                
                # # - backward
                # if file not in skip_bwd:
                #     tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd.n3")
                #     netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
                #     record_eye(times_file, run, file, data_file, 'bwd', 'create', netw_time1, reas_time1)
                #     netw_time2, reas_time2 = runNSave_timed(["eye", "--turtle", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file, max_time=max_time)
                #     record_eye(times_file, run, file, data_file, 'bwd', 'run', netw_time2, reas_time2)
                #     record_eye(times_file, run, file, data_file, 'bwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
                
                # - backward (auto re-ordering)
                if file not in skip_bwd:
                    tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd_auto.n3")
                    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "scores.n3", "--query", "rule-creation-bwd_opt.n3", "--nope", 
                                                        "--quantify", "http://www.w3.org/2000/10/swap/var"], tmp_file)
                    record_eye(times_file, run, file, data_file, 'bwd_auto', 'create', netw_time1, reas_time1)
                    netw_time2, reas_time2 = runNSave_timed(["eye", "--turtle", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file, max_time=max_time)
                    record_eye(times_file, run, file, data_file, 'bwd_auto', 'run', netw_time2, reas_time2)
                    record_eye(times_file, run, file, data_file, 'bwd_auto', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
                
                # # - backward (manual re-ordering)
                # for id in [ "ro1" ]:
                #     tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd_{id}.n3")
                #     netw_time1, reas_time1 = (0, 0)
                #     netw_time2, reas_time2 = runNSave_timed(["eye", "--turtle", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file, max_time=max_time)
                #     record_eye(times_file, run, file, data_file, f'bwd_{id}', 'run', netw_time2, reas_time2)
                #     record_eye(times_file, run, file, data_file, f'bwd_{id}', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
                    
                # # - forward
                # if file not in skip_fwd:
                #     tmp_file = os.path.join(path, "n3", "gen", f"{name}_fwd.n3")
                #     netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
                #     record_eye(times_file, run, file, data_file, 'fwd', 'create', netw_time1, reas_time1)
                #     netw_time2, reas_time2 = runNSave_timed(["eye", "--turtle", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file, max_time=max_time)
                #     record_eye(times_file, run, file, data_file, 'fwd', 'run', netw_time2, reas_time2)
                #     record_eye(times_file, run, file, data_file, 'fwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
            
                # # - mixed
                # if file not in skip_mxd:
                #     tmp_file = os.path.join(path, "n3", "gen", f"{name}_mxd.n3")
                #     netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation_mixed.n3", "--nope"], tmp_file)
                #     record_eye(times_file, run, file, data_file, 'mxd', 'create', netw_time1, reas_time1)
                #     netw_time2, reas_time2 = runNSave_timed(["eye", "--turtle", data_path, tmp_file, "--query", norm_file, "--nope"], mxd_res_file, max_time=max_time)
                #     record_eye(times_file, run, file, data_file, 'mxd', 'run', netw_time2, reas_time2)
                #     record_eye(times_file, run, file, data_file, 'mxd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
            
            except:
                print(">>error<<")
            
            times_file.flush()
        
        print()

setup 1_000_pt2
run: 0
pp1.n3 : 12:03:15
pp2-2.n3 : 12:03:15
pp2-re0.n3 : 12:03:16
pp2-re1.n3 : 12:03:16
pp2.n3 : 12:03:17
pp3.n3 : 12:03:18
pp4-2.n3 : 12:03:19
pp4-re0.n3 : 12:03:20
pp4-re1.n3 : 12:03:20
pp4.n3 : 12:03:21

setup 2_000_pt2
run: 0
pp1.n3 : 12:03:22
pp2-2.n3 : 12:03:22
pp2-re0.n3 : 12:03:23
pp2-re1.n3 : 12:03:24
pp2.n3 : 12:03:25
pp3.n3 : 12:03:27
pp4-2.n3 : 12:03:29
pp4-re0.n3 : 12:03:31
pp4-re1.n3 : 12:03:32
pp4.n3 : 12:03:33

setup 5_000_pt2
run: 0
pp1.n3 : 12:03:36
pp2-2.n3 : 12:03:38
pp2-re0.n3 : 12:03:40
pp2-re1.n3 : 12:03:42
pp2.n3 : 12:03:44
pp3.n3 : 12:03:55
pp4-2.n3 : 12:04:01
pp4-re0.n3 : 12:04:19
pp4-re1.n3 : 12:04:21
pp4.n3 : 12:04:26

setup 7_000_pt2
run: 0
pp1.n3 : 12:04:44
pp2-2.n3 : 12:04:47
pp2-re0.n3 : 12:04:49
pp2-re1.n3 : 12:04:52
pp2.n3 : 12:04:56
pp3.n3 : 12:05:17
pp4-2.n3 : 12:05:28
pp4-re0.n3 : 12:06:05
pp4-re1.n3 : 12:06:08
pp4.n3 : 12:06:18

setup 10_000_pt2
run: 0
pp1.n3 : 12:06:55
pp2-2.n3 : 12:07:00
pp2-re0.n3 : 12:07:03
pp2-re1.n3 : 12:07:0

Process Process-154:
Traceback (most recent call last):
  File "/Users/wvw/anaconda3/lib/python3.11/site-packages/multiprocess/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/wvw/anaconda3/lib/python3.11/site-packages/multiprocess/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_45451/3222760756.py", line 13, in __do_runNSave
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
                                              ^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wvw/anaconda3/lib/python3.11/subprocess.py", line 1209, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wvw/anaconda3/lib/python3.11/subprocess.py", line 2108, in _communicate
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/wvw/anaconda3/lib/python3.11/se

>>error<<
pp4-re0.n3 : 12:41:12
pp4-re1.n3 : 12:41:16


## Run SparqLog PP on Nemo

In [ ]:
path = "test"
in_path = os.path.join(path, "misc/test3.sparql")
out_path = os.path.join(path, "misc/test3_sparqlog.nmo")

run(["java", "-jar", "sparql2sparqlog.jar", in_path, out_path]) # translate
run(["/opt/nemo-0.6.0/nmo", out_path, "--export-dir", "results", "--overwrite-results"]) # run

## Run N3 PP on Nemo

In [ ]:
path = "test"
# substitute for rule file without collections
in_path = os.path.join(path, "data.n3") 
out_path = os.path.join(path, "data.nmo")

run(["java", "-jar", "n32nmo.jar", in_path, out_path]) # translate
# run(["/opt/nemo-0.6.0/nmo", out_path, "--export-dir", "results", "--overwrite-results"]) # run

## Compare results

In [ ]:
import logging, sys

def msg(test, msg, to):
    str = f"- {test}: {msg}"
    log(str, to)
    
def log(str, to):
    to.write(str + "\n")
    print(str)
    to.flush()

In [ ]:
def compare_results(file1, file2, test, to):
    process = subprocess.Popen(['java', '-jar', "compare_res.jar", file1, file2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.strip()
    
    # print(error)
    if out != "results are the same!":
        msg(test, out, to=to)

In [ ]:
# data_size = str(100)
# system = 'rdf4j'
# path = "../../gmark_50_new/mix"

data_size = "7_000_pt2"
system = 'jena'
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/SPIN-to-N3/property-paths/zika"

files = list(os.listdir(os.path.join(path, "results", system, data_size)))
files.sort()
for version in ('direct', 'fwd', 'bwd'):
# for version in ('fwd',):
    with open(f"cmp/{version}_{data_size}_out.txt", 'w') as logfile:
        log(f"> comparing '{version}'", to=logfile)
        
        for file in files:
            if not file.endswith(".n3"):
                continue
            
            print(file)
            
            file1 = os.path.join(path, "results", system, data_size, file)
            file2 = os.path.join(path, "n3", "results", data_size, version, file)
            
            if not os.path.isfile(file2):
                msg(file, "cannot find file2", to=logfile)
            else:
                # too slow for files sized 2MB
                # compare_results(file1, file2, test=file, to=logfile)
                
                with open(file1, "rb") as f1:
                    num1 = sum(1 for _ in f1)
                with open(file2, "rb") as f2:
                    num2 = sum(1 for _ in f2)
                    num2 = (0 if num2 <= 5 else num2 - 7) # (comments at top & bottom of file)
                if num1 != num2:
                    msg(file, f"different # results: {num1} (sparql) <> {num2} (n3)", to=logfile)
                # else:
                #     print(f"same # results: {num1} (sparql) == {num2} (n3)")
                    